In [1]:
import findspark

In [2]:
findspark.find()

'/home/hadoop/spark-2.4.8-bin-hadoop2.7'

In [3]:
findspark.init()

In [4]:
import pyspark
sc=pyspark.SparkContext(appName="SparkContext")
sc.version

'2.4.8'

# 가장 많이 함께 대출한 책 top5 추출

In [5]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
file_path="hdfs://master:9000/user/data1"
br_df=sqlContext.read.csv(file_path, header=True)
br_df.show()

+---------------+---------------+-------+-------+-----+--------------------+------+---------+
|             id|       itempair|  item1|  item2|count|                prob|rscore|input_day|
+---------------+---------------+-------+-------+-----+--------------------+------+---------+
|3402488-5220784|3402488,5220784|3402488|5220784| 7292| 6.93183831743789E-4|   0.0| 20200909|
|3402488-2645744|2645744,3402488|3402488|2645744| 3723| 0.10646877144818119|   0.0| 20200909|
|3402488-2646422|2646422,3402488|3402488|2646422| 2870| 0.09199307647926148|   0.0| 20200909|
|3402488-3419276|3402488,3419276|3402488|3419276| 2705| 0.08934469546835777|   0.0| 20200909|
|3402488-3249896|3249896,3402488|3402488|3249896| 2382|   0.108529250956807|   0.0| 20200909|
|3402488-3425349|3402488,3425349|3402488|3425349| 2326| 0.12134808013355593|   0.0| 20200909|
|3402488-3202934|3202934,3402488|3402488|3202934| 2000| 0.08880994671403197|   0.0| 20200909|
|3402488-2646389|2646389,3402488|3402488|2646389| 1949| 0.07

In [6]:
from pyspark.sql.types import IntegerType
br_df = br_df.withColumn("item1", br_df["item1"].cast(IntegerType()))
br_df = br_df.withColumn("item2", br_df["item2"].cast(IntegerType()))
br_df = br_df.withColumn("count", br_df["count"].cast(IntegerType()))
br_df.printSchema()
print(br_df.count())

root
 |-- id: string (nullable = true)
 |-- itempair: string (nullable = true)
 |-- item1: integer (nullable = true)
 |-- item2: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- prob: string (nullable = true)
 |-- rscore: string (nullable = true)
 |-- input_day: string (nullable = true)

605887


In [8]:
temp = br_df.groupby("item1", "item2").sum("count").withColumnRenamed("sum(count)", "count")
temp = temp.sort(temp["item1"])
temp.show()

+------+-------+-----+
| item1|  item2|count|
+------+-------+-----+
|117085|2374415| 1143|
|117085|3151708|  930|
|117085|3190615|  978|
|117085|2376187|  987|
|117085|2155671| 1356|
|117085|5220784| 6744|
|117085|3369923| 1215|
|117085|1902173| 1116|
|117085|2037399| 1107|
|117085|3451595|  984|
|117086|1323591|  620|
|117086|3369923|  316|
|117086|1039479|  312|
|117086|2272922|  616|
|117086|3423096|  343|
|117086|1902173|  675|
|117086|5220784| 4735|
|117086|2373275|  645|
|117086|1561464|  693|
|117086|2192267|  271|
+------+-------+-----+
only showing top 20 rows



# 책 상세정보에서 isbn얻어서 조인하기

In [20]:
file_path="hdfs://master:9000/user/data2"
bi_df=sqlContext.read.csv(file_path, header=True)
bi_df = bi_df.select(bi_df["master_seq"], bi_df["isbn13"])
bi_df = bi_df.withColumn("master_seq", bi_df["master_seq"].cast(IntegerType()))
bi_df.printSchema()

bi_df1 = bi_df.withColumnRenamed("master_seq", "master_seq_1").withColumnRenamed("isbn13", "isbn13_1")
bi_df2 = bi_df.withColumnRenamed("master_seq", "master_seq_2").withColumnRenamed("isbn13", "isbn13_2")
bi_df1.show()
bi_df2.show()

root
 |-- master_seq: integer (nullable = true)
 |-- isbn13: string (nullable = true)

+------------+-------------+
|master_seq_1|     isbn13_1|
+------------+-------------+
|     2501855|9788956145792|
|     2501856|9788956145808|
|     2501857|9788956146010|
|     2501858|9788956146027|
|     2501859|9780516250342|
|     2501860|9781599530383|
|     2501861|9780531125717|
|     2501862|9780531175682|
|     2501863|9780516248783|
|     2501864|9780761328346|
|     2501865|9780805006827|
|     2501866|9780836881691|
|     2501867|9781429600606|
|     2501868|9780531184929|
|     2501869|9780822574958|
|     2501870|9781404238039|
|     2501873|9788954809368|
|     2501881|9788956146089|
|     2501882|9788960043367|
|     2501886|9788956146034|
+------------+-------------+
only showing top 20 rows

+------------+-------------+
|master_seq_2|     isbn13_2|
+------------+-------------+
|     2501855|9788956145792|
|     2501856|9788956145808|
|     2501857|9788956146010|
|     2501858|978

In [22]:
left_temp1 = temp.join(bi_df1, temp["item1"] == bi_df1["master_seq_1"], how='left')
left_temp1 = left_temp1.drop("master_seq_1")

left_temp2 = left_temp1.join(bi_df2, left_temp1["item2"] == bi_df2["master_seq_2"], how='left')
left_temp2 = left_temp2.drop("master_seq_2")
left_temp2.show()

+-------+------+-----+-------------+-------------+
|  item1| item2|count|     isbn13_1|     isbn13_2|
+-------+------+-----+-------------+-------------+
|2527288|288473|   97|9788952765345|9788952723970|
|2527288|288473|   97|9788952765345|9788952723970|
|2527288|288473|   97|9788952765345|9788952723970|
|2527288|288473|   97|9788952765345|9788952723970|
|2527288|288473|   97|9788952765345|9788952723970|
|2527288|288473|   97|9788952765345|9788952723970|
|2527288|288473|   97|9788952765345|9788952723970|
|2527288|288473|   97|9788952765345|9788952723970|
| 179956|288473|  486|9788952723741|9788952723970|
| 179956|288473|  486|9788952723741|9788952723970|
| 179956|288473|  486|9788952723741|9788952723970|
| 179956|288473|  486|9788952723741|9788952723970|
|2429886|288473|  271|9788952763228|9788952723970|
|2429886|288473|  271|9788952763228|9788952723970|
|2429886|288473|  271|9788952763228|9788952723970|
|2429886|288473|  271|9788952763228|9788952723970|
|2072826|302855|    7|978045228

In [24]:
print(left_temp2.count())
left_temp2 = left_temp2.dropDuplicates()
left_temp2.show()

1503694
+-------+-------+-----+-------------+-------------+
|  item1|  item2|count|     isbn13_1|     isbn13_2|
+-------+-------+-----+-------------+-------------+
|2527288| 288473|   97|9788952765345|9788952723970|
| 179956| 288473|  486|9788952723741|9788952723970|
|2429886| 288473|  271|9788952763228|9788952723970|
|2072826| 302855|    7|9780452282155|9788982180576|
|3273060| 331184|    5|9791158672119|9788985494571|
| 340340| 331184| 1400|9788984280120|9788985494571|
| 906244| 378702|   11|9788989863366|9788937814341|
|1854280| 691054|  152|9788958625162|9788972911166|
|3251251| 691054|  176|9791186688519|9788972911166|
|2074865| 923176|    7|9780670715121|9780670451494|
|3040962| 923386|    5|9780689863554|9780698114005|
|2095874| 953170|  846|9788911026692|9788911026333|
| 953167| 953170|  988|9788911026326|9788911026333|
|2182556| 953170|  674|9788911027231|9788911026333|
|2017885|1018288|  706|9788969020109|9788991980709|
|1045263|1045270|  108|9788930240727|9788930240734|
|108

In [27]:
output = left_temp2
output = output.sort(output["item1"], ascending=True)
output = output.na.drop('any')
print(output.count())
output.show()

238737
+------+-------+-----+-------------+-------------+
| item1|  item2|count|     isbn13_1|     isbn13_2|
+------+-------+-----+-------------+-------------+
|117085|3151708|  930|9788937460005|9788956609959|
|117085|3190615|  978|9788937460005|9788954641630|
|117085|3369923| 1215|9788937460005|9788937473135|
|117085|2155671| 1356|9788937460005|9788936433598|
|117085|3451595|  984|9788937460005|9788954646079|
|117085|2376187|  987|9788937460005|9788937461033|
|117085|2374415| 1143|9788937460005|9788937460449|
|117085|2037399| 1107|9788937460005|9788936434120|
|117085|1902173| 1116|9788937460005|9788972756194|
|117086|3423096|  343|9788936433444|9788936434267|
|117086|2272922|  616|9788936433444|9788954607346|
|117086|1512043|  590|9788936433444|9788987721095|
|117086|1561464|  693|9788936433444|9788987721217|
|117086|1902173|  675|9788936433444|9788972756194|
|117086|1039479|  312|9788936433444|9788936456221|
|117086|2192267|  271|9788936433444|9788936433635|
|117086|1245351|  778|97

# 크롤링한 책 정보랑 결합하기

In [31]:
file_path="hdfs://master:9000/user/crawling_book_info/book_info.csv"
cbi_df=sqlContext.read.csv(file_path, header=True)
cbi_df = cbi_df.select(cbi_df["book_id"], cbi_df["book_isbn"])
cbi_df = cbi_df.na.drop('any')
print(cbi_df.count())
cbi_df.show()

cbi_df1 = cbi_df.withColumnRenamed("book_id", "book_id_1").withColumnRenamed("book_isbn", "book_isbn_1")
cbi_df2 = cbi_df.withColumnRenamed("book_id", "book_id_2").withColumnRenamed("book_isbn", "book_isbn_2")

9265
+--------+-------------+
| book_id|    book_isbn|
+--------+-------------+
|  182678|9788949110479|
| 2084345|9788970127248|
| 6050510|9788936442507|
| 6055863|9788954608657|
| 6348922|9788931006810|
| 6597874|9788956055152|
| 6793587|9780811877541|
| 6822225|9788983946966|
| 7820011|9788994361147|
| 8882146|9788972915812|
|10549475|9788960533875|
|11021275|9791187345817|
|11821551|9791158360375|
|12129712|9788962223774|
|12874120|9788964963548|
|13205543|9791188601042|
|13911289|9791130618654|
|14124667|9791133487660|
|14372679|9788931459623|
|14623284|9791160802030|
+--------+-------------+
only showing top 20 rows



In [33]:
output1 = output.join(cbi_df1, output["isbn13_1"] == cbi_df1["book_isbn_1"], how='left')
output2 = output1.join(cbi_df2, output1["isbn13_2"] == cbi_df2["book_isbn_2"], how='left')
output2 = output2.na.drop('any')
print(output2.count())
output2.show()

11679
+------+-------+-----+-------------+-------------+---------+-------------+---------+-------------+
| item1|  item2|count|     isbn13_1|     isbn13_2|book_id_1|  book_isbn_1|book_id_2|  book_isbn_2|
+------+-------+-----+-------------+-------------+---------+-------------+---------+-------------+
|126479|1902173|19275|9788982814471|9788972756194|    64853|9788982814471|  7102772|9788972756194|
|126479|3411733| 3496|9788982814471|9788998274795|    64853|9788982814471| 11611116|9788998274795|
|126479|3027835| 2948|9788982814471|9788934972464|    64853|9788982814471|  9780781|9788934972464|
|126479|2037399| 7101|9788982814471|9788936434120|    64853|9788982814471|  7615129|9788936434120|
|126479|2396926|11433|9788982814471|9788996991342|    64853|9788982814471|  8339910|9788996991342|
|126479|3151708| 6966|9788982814471|9788956609959|    64853|9788982814471| 10567640|9788956609959|
|126479|3423096| 4468|9788982814471|9788936434267|    64853|9788982814471| 11802243|9788936434267|
|126

In [40]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

output3 = output2.select("book_id_1", "book_isbn_1", "book_id_2", "book_isbn_2", "count")
output3.show()
print(output2.count())

window = Window.partitionBy(output3["book_id_1"]).orderBy(output3["count"].desc())
result = output3.select('*', rank().over(window).alias("rank")).filter(col("rank") <= 5)
result.show()
print(result.count())

+---------+-------------+---------+-------------+-----+
|book_id_1|  book_isbn_1|book_id_2|  book_isbn_2|count|
+---------+-------------+---------+-------------+-----+
|    64853|9788982814471|  7102772|9788972756194|19275|
|    64853|9788982814471| 11611116|9788998274795| 3496|
|    64853|9788982814471|  9780781|9788934972464| 2948|
|    64853|9788982814471|  7615129|9788936434120| 7101|
|    64853|9788982814471|  8339910|9788996991342|11433|
|    64853|9788982814471| 10567640|9788956609959| 6966|
|    64853|9788982814471| 11802243|9788936434267| 4468|
|    64853|9788982814471|  6586721|9788956604992| 7011|
|    64853|9788982814471| 11211133|9788937473135|15144|
|   172701|9788934908067|  2098049|9788990611260|16398|
|   172701|9788934908067| 11792225|9788936447014| 9280|
|   172701|9788934908067|   170132|9788901033518|15984|
|   172701|9788934908067|  8638831|9788934969952|82090|
|   172701|9788934908067|  6289560|9788949161341| 9216|
|   172701|9788934908067|   172781|9788935652792

In [39]:
result.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("book_borrow_top5.csv")